# 1. 라이브러리 설치하기,  불러오기

In [1]:
#라이브러리 호출
import cv2
import os
import pafy
#!pip install --upgrade youtube_dl (pafy 없다면)


In [2]:
!pip install --upgrade azure-cognitiveservices-vision-computervision
!pip install pillow

# 2. 영상 전처리 (이미지 캡쳐)

In [2]:
filepath = 'C:/Users/ansrl/Documents/이문기/11_TAVE/2022 AI_SPARK/비디오요약/tvsum50_ver_1_1/ydata-tvsum50-v1_1/tvsum50_video/_xMr-HKMfVA.mp4'
vidcap = cv2.VideoCapture()
vidcap.open(filepath)

fps = vidcap.get(cv2.CAP_PROP_FPS) 
frame_count = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
duration = frame_count / fps
print("영상의 길이 : ", duration, "초")

count = 0                # count 번째 사진
increase_width = 10        # 여기서 몇초마다 찍을건지 세팅하면 됌.
second = 0
success = True


영상의 길이 :  148.91543333333334 초


In [3]:
while success and second <= duration:
    success,image = vidcap.read()
    vidcap.set(cv2.CAP_PROP_POS_MSEC, second * 1000)
    print(second, "초 에서 캡쳐")
    cv2.imwrite("../images/frame%d.jpg" % count, image)
    # 왠지 모르겠는데 폴더 중에서 한글이 있으면 작동 안 하는 듯 (캡쳐가 안됨)
    print("saved frame%d.jpg" % count)
    count += 1   
    second += increase_width
    if cv2.waitKey(10) == 27:
        break
        
print('----- Finish Video Capture! -----')

0 초 에서 캡쳐
saved frame0.jpg
10 초 에서 캡쳐
saved frame1.jpg
20 초 에서 캡쳐
saved frame2.jpg
30 초 에서 캡쳐
saved frame3.jpg
40 초 에서 캡쳐
saved frame4.jpg
50 초 에서 캡쳐
saved frame5.jpg
60 초 에서 캡쳐
saved frame6.jpg
70 초 에서 캡쳐
saved frame7.jpg
80 초 에서 캡쳐
saved frame8.jpg
90 초 에서 캡쳐
saved frame9.jpg
100 초 에서 캡쳐
saved frame10.jpg
110 초 에서 캡쳐
saved frame11.jpg
120 초 에서 캡쳐
saved frame12.jpg
130 초 에서 캡쳐
saved frame13.jpg
140 초 에서 캡쳐
saved frame14.jpg
----- Finish Video Capture! -----


# 3. Azure Cognitive Service 이미지분석 불러오기

### ※ 참고사항!

#### 1) URL 기반으로 Description 만들 때 : Describe_image 메써드 사용
- Azure Cognitive Services Computer Vision SDK for Python : 이게 BEST
https://docs.microsoft.com/ko-kr/python/api/overview/azure/cognitiveservices-vision-computervision-readme?view=azure-python

- <참고만> : 오류뜸 https://docs.microsoft.com/ko-kr/azure/cognitive-services/computer-vision/quickstarts-sdk/image-analysis-client-library?tabs=visual-studio&pivots=programming-language-python

- <참고만> : 오류뜸 https://docs.microsoft.com/ko-kr/azure/cognitive-services/computer-vision/vision-api-how-to-topics/howtocallvisionapi?tabs=python


#### 2) 로컬 경로에서 Description 만들 때 : describe_image_in_stream 메써드 사용
https://github.com/Azure-Samples/cognitive-services-quickstart-code/blob/master/python/ComputerVision/ImageAnalysisQuickstart.py

### 1. 라이브러리 불러오기

In [4]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

### 2. 계정 인증하기

In [5]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''

#key와 엔드포인트는 azure portal에서 cognitive service를 만들고 나면 가져올 수 있음
#현재의 키와 엔드포인트는 내 계정인데, 다른 컴퓨터에서도 돌아갈지는 모르겠음

subscription_key = "9e14cf501b8b4ca98a27c3bb2391e803"
endpoint = "https://lmg1111111.cognitiveservices.azure.com/"

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

### 3. 이미지 가져오기

In [6]:
# Images used for the examples: Describe an image, Categorize an image, Tag an image, 
# Detect faces, Detect adult or racy content, Detect the color scheme, 
# Detect domain-specific content, Detect image types, Detect objects

import glob
#images_folder = os.path.join (os.path.dirname(os.path.abspath('')), "images")
image_path = sorted(glob.glob('../images/frame*.jpg'), key=os.path.getctime)
print(image_path)
# url로 불러오고자 할 때 아래
#remote_image_url = "https://moderatorsampleimages.blob.core.windows.net/samples/sample16.png"

'''
END - Quickstart variables
'''

['../images\\frame0.jpg', '../images\\frame1.jpg', '../images\\frame2.jpg', '../images\\frame3.jpg', '../images\\frame4.jpg', '../images\\frame5.jpg', '../images\\frame6.jpg', '../images\\frame7.jpg', '../images\\frame8.jpg', '../images\\frame9.jpg', '../images\\frame10.jpg', '../images\\frame11.jpg', '../images\\frame12.jpg', '../images\\frame13.jpg', '../images\\frame14.jpg']


'\nEND - Quickstart variables\n'

### 4. 분석하는 함수 정의

In [7]:
def azure_cv(path):
    '''
    Describe an Image - local
    This example describes the contents of an image with the confidence score.
    '''
    print("===== Describe an Image - local =====")
    # images 폴더에서 image2.png 파일을 불러온다
    #혹시나 해서 실험해 봤는데 png든 jpg든 결과에는 관계없다. 확장자에 따라 결과가 달라지지 않는다
    local_image_path = os.path.join(path)
    local_image = open(local_image_path, "rb")
    
    # Call API
    description_result = computervision_client.describe_image_in_stream(local_image)

    # Get the captions (descriptions) from the response, with confidence level
    print("Description of local image: ")
    if (len(description_result.captions) == 0):
        print("No description detected.")
    else:
        for caption in description_result.captions:
            print("'{}' : '{}' with confidence {:.2f}%".format(path, caption.text, caption.confidence * 100))
    print()
    return caption.text
    #text만 따로 뽑아쓰려면 caption[0].text로 뽑아쓸 수 있다
    #print(description_result.captions[0].text)
    '''
    END - Describe an Image - local
    '''

### 5. 분석하자!

In [8]:
Z = []
for path in image_path:
    feature = [azure_cv(path)]
    for ele in feature:
        Z.append(ele)

===== Describe an Image - local =====
Description of local image: 
'../images\frame0.jpg' : 'a cube with a black background' with confidence 27.32%

===== Describe an Image - local =====
Description of local image: 
'../images\frame1.jpg' : 'a cube with a black background' with confidence 27.32%

===== Describe an Image - local =====
Description of local image: 
'../images\frame2.jpg' : 'a close-up of a card' with confidence 42.50%

===== Describe an Image - local =====
Description of local image: 
'../images\frame3.jpg' : 'a sign with text' with confidence 25.38%

===== Describe an Image - local =====
Description of local image: 
'../images\frame4.jpg' : 'a group of people in a room' with confidence 48.67%

===== Describe an Image - local =====
Description of local image: 
'../images\frame5.jpg' : 'a group of people sitting in a room' with confidence 51.26%

===== Describe an Image - local =====
Description of local image: 
'../images\frame6.jpg' : 'a person playing a saxophone' with 

### 6. csv 파일로 출력

In [16]:
import pandas as pd
df_Z = pd.DataFrame(Z)
df_Z.to_csv('../images/azure_cv.csv', index=False)

# 4. TTS 가즈아

### ※ 참고사항!

#### 1. 텍스트 음성 변환
https://docs.microsoft.com/ko-kr/azure/cognitive-services/speech-service/get-started-text-to-speech?tabs=terminal&pivots=programming-language-python

#### 2. 텍스트 파일 저장
https://docs.microsoft.com/ko-kr/azure/cognitive-services/speech-service/how-to-speech-synthesis?tabs=script%2Cbrowserjs%2Cterminal&pivots=programming-language-python

### 1. 라이브러리 설치

In [22]:
pip install azure-cognitiveservices-speech

Note: you may need to restart the kernel to use updated packages.


### 2. 인증부터

In [9]:
import azure.cognitiveservices.speech as speechsdk


# 여기서도 key와 region은 portal에서 cognitive service를 만들면 뽑을 수 있다
speech_config = speechsdk.SpeechConfig(subscription="4273fb5b821a4785bc85836bea53d2a1", region="KoreaCentral")
audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

# 제니씨의 음성으로 들어보자
speech_config.speech_synthesis_voice_name='en-US-JennyNeural'

speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

### 3. 스피커 출력 + 파일 저장

In [10]:
for a in range(0, len(Z)):
    # Get text from the console and synthesize to the default speaker.
    print("Enter some text that you want to speak >")

    #text = input()을 넣으면 따로 내가 출력하고 싶은 글귀를 넣을 수도 있다
    #일단은 아까의 분석결과를 넣어보도록 하자
    text = Z[a]
    
    speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()

    #스피커로 출력하기
    if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized for text [{}]".format(text))
    elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_synthesis_result.cancellation_details
        print("Speech synthesis canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(cancellation_details.error_details))
                print("Did you set the speech resource key and region values?")
    audio_config = speechsdk.audio.AudioOutputConfig(filename='../images/%d.wav' % a)
    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
    synthesizer.speak_text_async(Z[a])

Enter some text that you want to speak >
Speech synthesized for text [a cube with a black background]
Enter some text that you want to speak >
Speech synthesized for text [a cube with a black background]
Enter some text that you want to speak >
Speech synthesized for text [a close-up of a card]
Enter some text that you want to speak >
Speech synthesized for text [a sign with text]
Enter some text that you want to speak >
Speech synthesized for text [a group of people in a room]
Enter some text that you want to speak >
Speech synthesized for text [a group of people sitting in a room]
Enter some text that you want to speak >
Speech synthesized for text [a person playing a saxophone]
Enter some text that you want to speak >
Speech synthesized for text [a person holding a baby]
Enter some text that you want to speak >
Speech synthesized for text [a group of people playing instruments]
Enter some text that you want to speak >
Speech synthesized for text [a person with a mask]
Enter some tex